### Day 24: Crossed Wires

Link: https://adventofcode.com/2024/day/24#part2

Note: I'm not sure if this approach works on all inputs.

To solve part two, given the expected behavior of binary addition through logic gates, we need some insights from the input:

1. All operations resulting in a "z" wire should use `XOR`, except for the most significant bit.
2. All `XOR` operations should either involve "x", "y", or "z" — the initial inputs and the final output, respectively.
3. The result of operations involving `AND`, except when `x00` is part of the input, cannot be used as input in an operation with an operand other than `OR`.
4. The result of operations involving `XOR` cannot be used as input in an operation with the `OR` operand.

We then collect all output wires that break any of these rules and use them as our answer.

In [17]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
from collections import deque
from dataclasses import dataclass


wires: dict[str, int] = {}


for idx, line in enumerate(lines):
    line = line.strip()

    if not line:
        break

    wire, value = line.split(": ")
    wires[wire] = int(value)


@dataclass
class Operation:
    input_wire_1: str
    input_wire_2: str
    operand: str
    output_wire: str


operations: deque[Operation] = deque()
operands = {
    "AND": lambda v1, v2: v1 & v2,
    "OR": lambda v1, v2: v1 | v2,
    "XOR": lambda v1, v2: v1 ^ v2,
}


for line in lines[idx + 1 :]:
    line = line.strip()
    command, output_wire = line.split(" -> ")
    input_wire_1, operand, input_wire_2 = command.split(" ")
    operations.append(
        Operation(
            input_wire_1=input_wire_1,
            input_wire_2=input_wire_2,
            operand=operand,
            output_wire=output_wire,
        )
    )


wrong_outputs: list[str] = []


for operation in operations:
    if (
        operation.output_wire[0] == "z"
        and operation.operand != "XOR"
        and operation.output_wire != "z45"
    ):
        wrong_outputs.append(operation.output_wire)
        continue

    if (
        operation.operand == "XOR"
        and operation.output_wire[0] != "z"
        and operation.input_wire_1[0] not in {"x", "y"}
        and operation.input_wire_2[0] not in {"x", "y"}
    ):
        wrong_outputs.append(operation.output_wire)
        continue

    if operation.operand == "AND" and "x00" not in {
        operation.input_wire_1,
        operation.input_wire_2,
    }:
        for next_operation in operations:
            if (
                operation.output_wire
                in {next_operation.input_wire_1, next_operation.input_wire_2}
                and next_operation.operand != "OR"
            ):
                wrong_outputs.append(operation.output_wire)
                break

        continue

    if operation.operand == "XOR":
        for next_operation in operations:
            if (
                operation.output_wire
                in {next_operation.input_wire_1, next_operation.input_wire_2}
                and next_operation.operand == "OR"
            ):
                wrong_outputs.append(operation.output_wire)
                break


print(",".join(sorted(wrong_outputs)))